In [9]:
import streamlit as st
import sys
print(sys.executable)
import seaborn as sns 

import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go  
from datetime import datetime, timedelta
from crypto_visualizations import perform_eda
from correlation_analysis import crypto_analysis
from random_forest_crypto import load_and_preprocess_data , run_random_forest_pipeline
from sarimax_forecasting import preprocess_crypto_data, train_sarimax
from lstm_forecasting_module import preprocess_data, run_lstm_forecast
from prophet_forecasting_module import preprocess_data as prophet_preprocess, train_and_forecast_prophet
from data_fetcher import get_crypto_data_realtime  
from crypto_news import get_crypto_news


def main():
    
    st.set_page_config(
        page_title="SOLIGENCE Cryptocurrency Analysis",
        page_icon="📈",
        layout="wide",
    )

    # CSS for styling
    st.markdown(
        """
        <style>
        /* App Background Image */
        .stApp {
            background-image: url("https://images6.alphacoders.com/982/982325.jpg");
            background-size: cover;
            background-repeat: no-repeat;
            background-position: center;
        }

        /* Lock-Style Menu Items */
        .stRadio > div {
            display: flex;
            flex-direction: column;
        }
        .stRadio div [data-baseweb="radio"] {
            background-color: transparent;
            border: none;
            color: white;
            font-size: 16px;
            font-weight: bold;
            margin: 5px 0;
            padding: 10px 15px;
            border-radius: 8px;
            cursor: pointer;
            transition: background-color 0.3s, color 0.3s;
        }
        .stRadio div [data-baseweb="radio"]:hover {
            background-color: #6c5ce7;
        }
        .stRadio div [data-baseweb="radio"] input:checked + div {
            background-color: #6c5ce7;
            color: white;
        }

        /* Welcome Menu Styling */
        .welcome-menu {
            text-align: center;
            margin-top: 100px;
            color: white;
        }
        </style>
        """,
        unsafe_allow_html=True,
    )

    # Sidebar Menu Options
    menu_options = [
        "🏠 Home",
        "📜 Historical Data Viewer",
        "🔍 Correlation Analysis",
        "📉 Data Visualizations",
        "📊 Random Forest Forecasting",
        "📈 SARIMAX Forecasting",
        "🤖 LSTM Forecasting",
        "📅 Prophet Forecasting",
        "📰 Cryptocurrency News",
        
    ]

    # Sidebar  Menu
    selected_menu = st.sidebar.radio("Navigate to:", menu_options)

    # Predefined cryptocurrencies
    allowed_cryptos = ["LTC-USD", "BTC-USD", "ETH-USD", "BCH-USD"]

    # Home Page
    if selected_menu == "🏠 Home":
        st.markdown(
            """
            <div class="welcome-menu">
                <h1 style="font-size: 60px; color: #FFD700;">Welcome to SOLIGENCE</h1>
                <h2 style="font-size: 30px; color: #FFFFFF;">Your Intelligent Cryptocurrency Trading Platform</h2>
                <p style="font-size: 18px; color: #FFFFFF;">
                    Analyze, Predict, and Visualize Cryptocurrency Data
                </p>
            </div>
            """,
            unsafe_allow_html=True,
        )

    # Historical Data Viewer
    elif selected_menu == "📜 Historical Data Viewer":
        st.header("Cryptocurrency Historical Data Viewer")
        st.write("Select a cryptocurrency and specify a date range to view historical data.")

        # Select cryptocurrency
        crypto = st.selectbox("Choose a cryptocurrency:", allowed_cryptos)

        # Date range slider
        today = datetime.today().date()
        one_year_back = today - timedelta(days=365)

        start_date, end_date = st.slider(
            "Select Date Range",
            min_value=one_year_back,
            max_value=today,
            value=(one_year_back, today),
            format="YYYY-MM-DD",
        )
        st.write(f"Selected Date Range: {start_date} to {end_date}")

        # Fetch historical data
        if st.button("View Historical Data"):
            try:
                # Fetch and preprocess data
                historical_data = get_crypto_data_realtime(
                    crypto_name=crypto,
                    start_date=start_date.strftime("%Y-%m-%d"),
                    end_date=end_date.strftime("%Y-%m-%d"),
                )

                if historical_data.empty:
                    st.warning(f"No data found for {crypto} in the selected range ({start_date} to {end_date}).")
                else:
                    # Display Cleaned Data
                    st.subheader(f"Cleaned Data for {crypto}")
                    st.dataframe(historical_data)

                    # Line Chart of Prices
                    st.subheader("Price Chart with Moving Averages")
                    fig = go.Figure()
                    fig.add_trace(go.Scatter(x=historical_data["Date"], y=historical_data["close"],
                                             mode="lines", name="Close Price"))
                    fig.add_trace(go.Scatter(x=historical_data["Date"], y=historical_data["7-Day MA"],
                                             mode="lines", name="7-Day MA"))
                    fig.add_trace(go.Scatter(x=historical_data["Date"], y=historical_data["14-Day MA"],
                                             mode="lines", name="14-Day MA"))
                    fig.add_trace(go.Scatter(x=historical_data["Date"], y=historical_data["30-Day MA"],
                                             mode="lines", name="30-Day MA"))

                    fig.update_layout(
                        title=f"Price and Moving Averages for {crypto}",
                        xaxis_title="Date",
                        yaxis_title="Price (USD)",
                        template="plotly_white",
                    )
                    st.plotly_chart(fig, use_container_width=True)

                    # Candlestick Chart
                    st.subheader("Candlestick Chart")
                    fig = go.Figure(data=[go.Candlestick(
                        x=historical_data["Date"],
                        open=historical_data["open"],
                        high=historical_data["high"],
                        low=historical_data["low"],
                        close=historical_data["close"],
                        increasing_line_color="green",
                        decreasing_line_color="red",
                    )])

                    fig.update_layout(
                        title=f"Candlestick Chart for {crypto}",
                        xaxis_title="Date",
                        yaxis_title="Price (USD)",
                        xaxis_rangeslider_visible=True,
                        template="plotly_white",
                    )

                    st.plotly_chart(fig, use_container_width=True)

            except ValueError as ve:
                st.error(f"Validation Error: {ve}")
            except Exception as e:
                st.error(f"An error occurred: {e}")





    elif selected_menu == "🔍 Correlation Analysis":
        st.header("Cryptocurrency Correlation Analysis")
        file_path = "transformed_cryptocurrency_data.csv"
        allowed_cryptos = ["LTC-USD", "BTC-USD", "ETH-USD", "BCH-USD"]
        
        
        
        
        
        st.subheader("Clustering Results")
        try:
            results = crypto_analysis(file_path, n_components=10, n_clusters=4, selected_cryptos=allowed_cryptos)
            st.write(f"**Silhouette Score:** {results['silhouette_score']:.2f}")
            st.write("### Random Cryptocurrencies from Each Cluster:")
            st.table(results['random_crypto_table'])
            
            st.subheader("Correlation Matrix for Random Cryptocurrencies")
            random_cryptos = results['random_crypto_table']['Cryptocurrency'].tolist()
            correlation_matrix_random = results['correlation_matrix'].loc[random_cryptos, random_cryptos]
            
            fig, ax = plt.subplots(figsize=(8, 6))
            sns.heatmap(
                correlation_matrix_random,
                
                
                annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5,
                cbar=True, ax=ax
            )
        
            ax.set_title("Correlation Heatmap for Random Cryptocurrencies", fontsize=14)
            plt.xticks(rotation=45, ha='right')
            plt.yticks(rotation=0)
            st.pyplot(fig)

            
            st.write("### Select a cryptocurrency to view correlations:")
            selected_crypto = st.selectbox("Choose a cryptocurrency:", allowed_cryptos)
            if selected_crypto:
                st.subheader(f"Correlation Analysis for {selected_crypto}")
                selected_correlations = results['random_crypto_correlations'].get(selected_crypto, None)
                if selected_correlations:
                    st.write("Top Positive Correlations:")
                    pos_corr = selected_correlations['Top Positive'].reset_index().rename(columns={'index': 'Symbol'})
                    st.table(pos_corr)
                    st.write("Top Negative Correlations:")
                    neg_corr = selected_correlations['Top Negative'].reset_index().rename(columns={'index': 'Symbol'})
                    st.table(neg_corr)
                else:
                    st.write("No correlation data available for this cryptocurrency.")
            st.subheader("Global Cryptocurrency Correlation Heatmap")
            st.pyplot(results['global_correlation_fig'])
        except Exception as e:
            st.error(f"An error occurred: {e}")
        
        
            
                
            
        
        

                    
                
                


    elif selected_menu == "📊 Random Forest Forecasting":
        st.header("Cryptocurrency Forecasting with Random Forest")
        file_path = "reshaped_crypto_data_wide.csv"

        st.write("### Select a Cryptocurrency for Analysis")
        crypto_symbol = st.selectbox("Choose a cryptocurrency:", allowed_cryptos)

        if st.button("Run Random Forest Forecast"):
            try:
                
                processed_data = load_and_preprocess_data(
                file_path, crypto_symbol, 
                ['Adj', 'Open', 'High', 'Low', 'Volume', 
                 ]
                )
            
                accuracy, fig, market_state_df = run_random_forest_pipeline(processed_data, crypto_symbol)
               
                

                st.subheader(f"Forecasting Results for {crypto_symbol}")
                st.subheader(f"Model Accuracy for {crypto_symbol}")
                st.write(f"**Accuracy:** {accuracy:.2f}%")
                st.plotly_chart(fig, use_container_width=True)
                st.subheader("Market State Predictions")
                st.dataframe(market_state_df)
                
                
            except Exception as e:
                st.error(f"An error occurred: {e}")

    elif selected_menu == "📈 SARIMAX Forecasting":
        st.header("Cryptocurrency Forecasting with SARIMAX")
        file_path = "reshaped_crypto_data_wide.csv"

        st.write("### Select a Cryptocurrency for Analysis")
        crypto_symbol = st.selectbox("Choose a cryptocurrency:", allowed_cryptos)

        if st.button("Run SARIMAX Forecast"):
            try:
                st.info("Preprocessing data...")
                crypto_series = preprocess_crypto_data(file_path, crypto_symbol)
                st.success("Data preprocessing completed!")
                st.info("Training SARIMAX model...")
                accuracy, fig,sarimax_forecast = train_sarimax(crypto_series, crypto_symbol)
                
                st.success(f"Model training completed for {crypto_symbol}!")
                st.subheader(f"Results for {crypto_symbol}")
                st.subheader(f"Model Accuracy for {crypto_symbol}")
                st.write(f"**Accuracy:** {accuracy:.2f}%")

                # Display forecast chart
                st.subheader("SARIMAX Forecast Chart")
                st.plotly_chart(fig, use_container_width=True)
            except Exception as e:
                st.error(f"An error occurred: {e}")

    elif selected_menu == "🤖 LSTM Forecasting":
        st.header("Cryptocurrency Forecasting with LSTM")
        file_path = "reshaped_crypto_data_wide.csv"

        st.write("### Select a Cryptocurrency for Analysis")
        crypto_symbol = st.selectbox("Choose a cryptocurrency:", allowed_cryptos)

        if st.button("Run LSTM Forecast"):
            try:
                st.info("Preprocessing data...")
                scaled_data, scaler, crypto_series = preprocess_data(file_path, crypto_symbol, ["Adj", "Open", "High", "Low", "Volume"])
                st.success("Data preprocessing completed!")
                st.info("Training LSTM model...")
                predictions, actual, accuracy, high_low_df, fig = run_lstm_forecast(scaled_data, scaler, crypto_series)
            
                st.success("Model training and evaluation completed!")

                st.subheader(f"Model Accuracy for {crypto_symbol}")
                st.write(f"**Accuracy:** {accuracy:.2f}%")
                st.plotly_chart(fig, use_container_width=True)
                st.subheader(f"Predicted Highs and Lows for {crypto_symbol}")
                st.write(high_low_df)
            

                
            except Exception as e:
                st.error(f"An error occurred: {e}")

    elif selected_menu == "📅 Prophet Forecasting":
        st.header("Cryptocurrency Forecasting with Prophet")
        file_path = "reshaped_crypto_data_wide.csv"

        st.write("### Select a Cryptocurrency for Analysis")
        crypto_symbol = st.selectbox("Choose a cryptocurrency:", allowed_cryptos)

        if st.button("Run Prophet Forecast"):
            try:
                st.info("Preprocessing data...")
                train, test,crypto_series = prophet_preprocess(file_path, crypto_symbol)
                st.success("Data preprocessing completed!")
                progress_bar = st.progress(0)
                st.info("Training Prophet model...")
                for progress in range(1,101,25):
                    progress_bar.progress(progress)
                    st.write(f"Progress: {progress}%")
                accuracy, forecast, prophet_model,fig = train_and_forecast_prophet(train, test,crypto_series)
                progress_bar.progress(100)
                st.success("Model training and evaluation completed!")

                st.subheader(f"Model Accuracy for {crypto_symbol}")
                st.write(f"**Accuracy:** {accuracy:.2f}%")

                st.subheader(" Prophet Forecast Chart")
                st.plotly_chart(fig, use_container_width=True)
            except Exception as e:
                st.error(f"An error occurred: {e}")

    elif selected_menu == "📉 Data Visualizations":
        st.header("Cryptocurrency Data Visualizations")
        file_path = "reshaped_crypto_data_wide.csv"

        selected_cryptos = st.multiselect(
            "Select cryptocurrencies for visualization:", options=["LTC-USD", "BTC-USD", "ETH-USD", "BCH-USD"]
        )
            
        if st.button("Generate Visualizations"):
            if selected_cryptos:
                try:
                    eda_plots = perform_eda(file_path, selected_cryptos)
                    st.subheader("Temporal Structure: Price Trends Over Time")
                    st.plotly_chart(eda_plots['temporal_structure'], use_container_width=True)
                    
                    st.subheader("Distribution Visualization: KDE Plots")
                    st.plotly_chart(eda_plots['distribution'], use_container_width=True)
                    
                    st.subheader("Rolling Volatility Analysis (7-Day)")
                    st.plotly_chart(eda_plots['volatility'], use_container_width=True)
                                 
                    for crypto in selected_cryptos:
                        st.subheader(f" Candlestick Chart for {crypto}")
                        st.plotly_chart(eda_plots[f'candlestick_chart_{crypto}'], use_container_width=True)
                    for crypto in selected_cryptos:
                        st.subheader(f" Seasonal Decomposition of {crypto}")
                        st.plotly_chart(eda_plots[f'seasonal_decomposition_{crypto}'], use_container_width=True)

                    st.subheader(" Daily Returns Distribution")
                    st.plotly_chart(eda_plots['daily_returns'], use_container_width=True)
                        
                

                except Exception as e:
                    st.error(f"An error occurred: {e}")
            else:
                st.error("Please select at least one cryptocurrency.")
    
    elif selected_menu == "📰 Cryptocurrency News":
        
        st.title("Cryptocurrency News Dashboard")
        
        if st.button("Get Latest Crypto News"):
            
            news_articles = get_crypto_news()
            if news_articles:
            
                st.subheader("Latest Cryptocurrency News")
                for article in news_articles:
                    
                    st.write(f"**{article['title']}**")
                    st.write(f"Published At: {article['published_at']}")
                    st.write(f"Source: {article['source']}")
                    st.write(f"[Read More]({article['url']})")
                    st.write("---")
            
            else:
                
                st.write("No news articles found.")
                

   

if __name__ == "__main__":
    
    main()


                
            


                    
                






/opt/anaconda3/envs/COM724/bin/python


ModuleNotFoundError: No module named 'crypto_visualizations'